In [4]:
import MeCab
import pickle
import re
from collections import Counter

# MeCabの初期化（分かち書き用）
mecab = MeCab.Tagger("-Owakati")

# ファイルパス
fp_comments = 'comments.txt'
fp_word_to_id = 'word_to_id.pkl'
fp_id_to_word = 'id_to_word.pkl'

# comments.txt の読み込み
with open(fp_comments, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 文部分のみ抽出（index:文 → 文）
comments = [re.sub(r'^\d+:', '', line).strip() for line in lines]

# 分かち書きしてトークンリストを作成
tokens = []
for comment in comments:
    words = mecab.parse(comment).strip().split()  # 分かち書き実行
    tokens.extend(words)

# 句読点などの不要文字を削除
table = str.maketrans({'.': '', ',': '', '。': '', '、': '', '「': '', '」': '', '（': '', '）': ''})
tokens = [word.translate(table) for word in tokens]

# 単語の頻度カウント
freq = Counter(tokens)

# 3回以上出現する単語のみ抽出
vocab_kif = [token for token, count in freq.items() if count >= 3]
vocab_kif.sort()

# 特殊トークンの追加
vocab_kif.extend(['<start>', '<end>', '<unk>', '<null>'])

# 単語IDマッピングの作成
word_to_id = {token: i for i, token in enumerate(vocab_kif)}
id_to_word = {i: token for i, token in enumerate(vocab_kif)}

# 辞書ファイルの保存
with open(fp_word_to_id, 'wb') as f:
    pickle.dump(word_to_id, f)
with open(fp_id_to_word, 'wb') as f:
    pickle.dump(id_to_word, f)

print(f'単語数: {len(word_to_id)}')

単語数: 5256


In [6]:
import cshogi

In [8]:
import re

pattern = r"""
    ([１-９1-9一二三四五六七八九]{2})?      # 移動先
    (?P<same>同)?                                # 同
    \s*                                         # 空白文字
    (?P<piece>成[銀桂香]|[王玉金銀全桂圭香杏角馬飛龍竜歩と])  # 駒
    (?P<suffix>[左右直]?[寄引上行]?)               # 駒の動きの接尾辞
    (?P<motion>不?成|打|合|生)?                   # 動きの種類
"""

# テスト用の文字列
test_string = "２二同歩成"

# 正規表現をコンパイルして、テスト文字列にマッチさせる
match = re.match(pattern, test_string, re.VERBOSE)

if match:
    print(match.groupdict())
else:
    print("マッチしませんでした")


{'to': '２二', 'same': '同', 'piece': '歩', 'suffix': '', 'motion': '成', 'from': None}


In [13]:
import re

pattern = r"""
    [▲△]                           # 手番（▲または△）
    \d{1,2}\s?\d{1,2}?              # 移動元（オプション）
    同?                              # 同
    (歩|銀|金|角|飛|歩と|銀と|金と|桂|香|角成|飛成|竜|馬)?  # 駒
    (不成|打|成)?                    # 駒の動き（不成、打、成）
    \d{1,2}\s?\d{1,2}                 # 移動先
    [左右直寄引上行]?                # 駒の動きの接尾辞
"""

# テスト用の文字列
test_string = "▲4五歩△同歩▲3三歩不成△7七歩成▲同銀左△7六金打"

# 正規表現をコンパイルして、テスト文字列にマッチさせる
matches = re.findall(pattern, test_string, re.VERBOSE)

# 一致する部分を出力
for match in matches:
    print(match)


In [19]:
import re
import MeCab

def tokenize_shogi_text(text, pattern):
    # 駒の動きを一度に取得し、文中にスペースを挿入
    shogi_moves = re.findall(pattern, text)
    for move in shogi_moves:
        text = text.replace(move, f" {move} ")

    # MeCabによる分かち書き
    mecab = MeCab.Tagger("-Owakati")
    tokens = mecab.parse(text).strip().split()
    
    # 駒の動きを結合する処理
    combined_tokens = []
    i = 0
    while i < len(tokens):
        # 駒の動きに該当する部分を結合
        if re.match(r"^[▲△]\d{1,2}[一二三四五六七八九十]+同(桂|香|銀|金|角|飛|歩|と|成香|成桂|成銀|角成|飛成|竜|馬)?", tokens[i]):
            combined_move = tokens[i]
            i += 1
            while i < len(tokens) and not re.match(r"^[▲△]\d{1,2}[一二三四五六七八九十]+同", tokens[i]):
                combined_move += tokens[i]
                i += 1
            combined_tokens.append(combined_move)
        else:
            combined_tokens.append(tokens[i])
            i += 1

    return combined_tokens

# テスト
text = "▲1七同桂は△1八歩▲同香△4五角がある。"
pattern = r"""
    [▲△]                           # 手番（▲または△）
    \d{1,2}\s?\d{1,2}?              # 移動元（オプション）
    同                              # 同
    (桂|香|銀|金|角|飛|歩|と|成香|成桂|成銀|角成|飛成|竜|馬)?  # 駒
    (不成|打|成)?                    # 駒の動き（不成、打、成）
    \d{1,2}\s?\d{1,2}                 # 移動先
    [左右直寄引上行]?                # 駒の動きの接尾辞
"""
tokens = tokenize_shogi_text(text, pattern)
print(tokens)


['▲', '1', '七', '同桂', 'は', '△', '1', '八', '歩', '▲', '同香', '△', '4', '五角', 'が', 'ある', '。']


In [16]:
import re
import MeCab

def tokenize_shogi_text(text, pattern):
    # 駒の動きを一度に取得し、文中にスペースを挿入
    shogi_moves = re.findall(pattern, text)
    for move in shogi_moves:
        text = text.replace(move, f" {move} ")

    # MeCabによる分かち書き
    mecab = MeCab.Tagger("-Owakati")
    tokens = mecab.parse(text).strip().split()
    
    # 駒の動きを結合する処理
    combined_tokens = []
    i = 0
    while i < len(tokens):
        # 駒の動きに該当する部分を結合
        if re.match(r"^[▲△]\d{1,2}[一二三四五六七八九十]+同(桂|香|銀|金|角|飛|歩|と|成香|成桂|成銀|角成|飛成|竜|馬)?", tokens[i]):
            combined_move = tokens[i]
            i += 1
            while i < len(tokens) and not re.match(r"^[▲△]\d{1,2}[一二三四五六七八九]+同", tokens[i]):
                combined_move += tokens[i]
                i += 1
            combined_tokens.append(combined_move)
        else:
            combined_tokens.append(tokens[i])
            i += 1

    return combined_tokens

# テスト
text = "▲1七同桂は△1八歩▲同香△4五角がある。"
pattern = r"""
    [▲△]                           # 手番（▲または△）
    \d{1,2}\s?\d{1,2}?              # 移動元（オプション）
    同                              # 同
    (桂|香|銀|金|角|飛|歩|と|成香|成桂|成銀|角成|飛成|竜|馬)?  # 駒
    (不成|打|成)?                    # 駒の動き（不成、打、成）
    \d{1,2}\s?\d{1,2}                 # 移動先
    [左右直寄引上行]?                # 駒の動きの接尾辞
"""
tokens = tokenize_shogi_text(text, pattern)
print(tokens)


['▲', '1', '七', '同桂', 'は', '△', '1', '八', '歩', '▲', '同香', '△', '4', '五角', 'が', 'ある', '。']


In [11]:
import re

text = "▲1七同桂は△1八歩▲同香△4五角がある。"

pattern = r"([▲△])(\d{1,2}[一二三四五六七八九十]+)([同]?)(([桂香銀金角飛歩と成香成桂成銀竜馬]?(不成|打|成)?[左右直寄引上行]?)(\d{1,2}[一二三四五六七八九十]+[左右直寄引上行]?)?)"

matches = re.findall(pattern, text)

print(matches)

[('▲', '1七', '同', '桂', '桂', '', ''), ('△', '1八', '', '歩', '歩', '', ''), ('△', '4五', '', '角', '角', '', '')]


In [12]:
import re
import MeCab

def tokenize_shogi_text(text):
    """将棋棋譜のテキストをトークン化します。

    Args:
        text (str): 将棋棋譜のテキスト

    Returns:
        list: トークン化された指し手のリスト
    """

    pattern = r"([▲△])(\d{1,2}[一二三四五六七八九十]+)([同]?)(([桂香銀金角飛歩と成香成桂成銀竜馬]?(不成|打|成)?[左右直寄引上行]?)(\d{1,2}[一二三四五六七八九十]+[左右直寄引上行]?)?)"
    matches = re.findall(pattern, text)

    return matches

# テスト
text = "▲1七同桂は△1八歩▲同香△4五角がある。"
tokens = tokenize_shogi_text(text)

for token in tokens:
    print(token)

('▲', '1七', '同', '桂', '桂', '', '')
('△', '1八', '', '歩', '歩', '', '')
('△', '4五', '', '角', '角', '', '')


In [20]:
import re
import MeCab

def tokenize_shogi_text(text):
    """将棋棋譜のテキストを自然な日本語で分かち書きします。

    Args:
        text (str): 将棋棋譜のテキスト

    Returns:
        list: 分かち書きされた単語のリスト
    """

    pattern = r"([▲△])(\d{1,2}[一二三四五六七八九]+)([同]?)(([玉桂香銀金角飛歩と成香成桂成銀竜馬]?(不成|打|成)?[左右直寄引上行]?)(\d{1,2}[一二三四五六七八九]+[左右直寄引上行]?)?)"
    matches = re.findall(pattern, text)

    mecab = MeCab.Tagger("-Owakati")

    result = []
    for match in matches:
        result.extend(mecab.parse(match[0] + match[1] + match[2] + match[3]).strip().split())

    # 残りのテキストを分かち書き
    rest_text = re.sub(pattern, '', text).strip()
    result.extend(mecab.parse(rest_text).strip().split())

    return result

# テスト
text = "▲1七同桂は△1八歩▲同香△4五角がある。"
tokens = tokenize_shogi_text(text)
print(tokens)

['▲', '1', '七', '同桂', '△', '1', '八', '歩', '△', '4', '五角', 'は', '▲', '同香', 'が', 'ある', '。']


In [22]:
import MeCab

# MeCabのセットアップ
mecab = MeCab.Tagger("-Owakati")

# 文章
text = "▲1七同桂は△1八歩▲同香△4五角がある。"

# 単語の分割（分かち書き）
tokens = mecab.parse(text).strip().split()
print(tokens)

['▲', '1', '七', '同桂', 'は', '△', '1', '八', '歩', '▲', '同香', '△', '4', '五角', 'が', 'ある', '。']


In [23]:
import re
import MeCab

def tokenize_shogi_text(text, pattern):
    # 駒の動きを先に結合
    shogi_moves = re.findall(pattern, text)
    for move in shogi_moves:
        text = text.replace(move, f" {move} ")

    # MeCabによる分かち書き
    mecab = MeCab.Tagger("-Owakati")
    tokens = mecab.parse(text).strip().split()
    
    # 駒の動きを結合する処理
    combined_tokens = []
    i = 0
    while i < len(tokens):
        # 駒の動きに該当する部分を結合
        if re.match(r"^[▲△]\d{1,2}[一二三四五六七八九十]+同(桂|香|銀|金|角|飛|歩|と|成香|成桂|成銀|角成|飛成|竜|馬)?", tokens[i]):
            combined_move = tokens[i]
            i += 1
            while i < len(tokens) and not re.match(r"^[▲△]\d{1,2}[一二三四五六七八九十]+同", tokens[i]):
                combined_move += tokens[i]
                i += 1
            combined_tokens.append(combined_move)
        else:
            combined_tokens.append(tokens[i])
            i += 1

    return combined_tokens

# 文章
text = "▲1七同桂は△1八歩▲同香△4五角がある。"

# 駒の動きを結合するための正規表現
pattern = r"""
    [▲△]                           # 手番（▲または△）
    \d{1,2}\s?\d{1,2}?              # 移動元（オプション）
    同                              # 同
    (桂|香|銀|金|角|飛|歩|と|成香|成桂|成銀|角成|飛成|竜|馬)?  # 駒
    (不成|打|成)?                    # 駒の動き（不成、打、成）
    \d{1,2}\s?\d{1,2}                 # 移動先
    [左右直寄引上行]?                # 駒の動きの接尾辞
"""

# トークン化
tokens = tokenize_shogi_text(text, pattern)
print(tokens)


['▲', '1', '七', '同桂', 'は', '△', '1', '八', '歩', '▲', '同香', '△', '4', '五角', 'が', 'ある', '。']


In [24]:
import re
import MeCab

def tokenize_shogi_text(text, pattern):
    # MeCabによる分かち書き
    mecab = MeCab.Tagger("-Owakati")
    tokens = mecab.parse(text).strip().split()
    
    # 駒の動きを結合する処理
    combined_tokens = []
    i = 0
    while i < len(tokens):
        # 駒の動きに該当する部分を結合（▲1七同桂など）
        if re.match(r"^[▲△]\d{1,2}[一二三四五六七八九十]+同(桂|香|銀|金|角|飛|歩|と|成香|成桂|成銀|角成|飛成|竜|馬)?", tokens[i]):
            combined_move = tokens[i]
            i += 1
            while i < len(tokens) and not re.match(r"^[▲△]\d{1,2}[一二三四五六七八九十]+同", tokens[i]):
                combined_move += tokens[i]
                i += 1
            combined_tokens.append(combined_move)
        else:
            combined_tokens.append(tokens[i])
            i += 1

    return combined_tokens

# 文章
text = "▲1七同桂は△1八歩▲同香△4五角がある。"

# 駒の動きに該当する正規表現（駒の動きを検出）
pattern = r"""
    [▲△]                           # 手番（▲または△）
    \d{1,2}\s?\d{1,2}?              # 移動元（オプション）
    同                              # 同
    (桂|香|銀|金|角|飛|歩|と|成香|成桂|成銀|角成|飛成|竜|馬)?  # 駒
    (不成|打|成)?                    # 駒の動き（不成、打、成）
    \d{1,2}\s?\d{1,2}                 # 移動先
    [左右直寄引上行]?                # 駒の動きの接尾辞
"""

# トークン化
tokens = tokenize_shogi_text(text, pattern)
print(tokens)


['▲', '1', '七', '同桂', 'は', '△', '1', '八', '歩', '▲', '同香', '△', '4', '五角', 'が', 'ある', '。']


In [35]:
import re

# テストテキスト
text = "▲4五歩△同歩▲3三歩不成△7七歩成▲同銀左△7六金打"

# 正規表現を使って駒の動き部分を抽出
pattern = r"""
    (▲|△)                               # 手番（▲または△）
    (                                     # グループ開始
        (1|2|3|4|5|6|7|8|9)        # 数字（１〜９）
        (一|二|三|四|五|六|七|八|九)    # 漢数字（いち、に、さん、し、ご、ろく、しち、はち、きゅう）
        | 同                               # 同
        | (同){1,2}                        # 同（1回または2回）
    )                                     # グループ終了
    (王|玉|飛|龍|竜|角|馬|金|銀|成銀|桂|成桂|香|成香|歩|と)  # 駒の種類
    (左|右)?                             # 駒の動き方向（左、右）
    (上|直|引|寄)?                       # 駒の動きの接尾辞（上、直、引、寄）
    (成|不成|打)?                        # 駒の動きの詳細（成、不成、打）
"""


# 正規表現にマッチする部分を抽出
matches = re.findall(pattern, text, re.VERBOSE)

# 結果を表示
print(matches)


[('▲', '4五', '4', '五', '', '歩', '', '', ''), ('△', '同', '', '', '', '歩', '', '', ''), ('▲', '3三', '3', '三', '', '歩', '', '', '不成'), ('△', '7七', '7', '七', '', '歩', '', '', '成'), ('▲', '同', '', '', '', '銀', '左', '', ''), ('△', '7六', '7', '六', '', '金', '', '', '打')]
